This is an open-source dataset created by the Teradata Center at Duke University. The Cell2Cell dataset is pre-processed and contains over 70,000 instances and 58 attributes. It can be used to understand subscriber churn. Some interesting questions to explore while solving this problem are:

Is the company losing high-value subscribers?
What different customer segmentations are possible? Which ones can help us analyze the churn problem?
How well are the retention policies implemented? What role does customer support play in minimizing churn?

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold
import json

In [2]:
# Read data
df = pd.read_csv(r'data/cell2celltrain.csv')

In [3]:
df.head()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,...,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,3000002,Yes,24.00,219.0,22.0,0.25,0.0,0.0,-157.0,-19.0,...,0,4,No,0,30,Yes,1-Highest,Suburban,Professional,No
1,3000010,Yes,16.99,10.0,17.0,0.00,0.0,0.0,-4.0,0.0,...,0,5,No,0,30,No,4-Medium,Suburban,Professional,Yes
2,3000014,No,38.00,8.0,38.0,0.00,0.0,0.0,-2.0,0.0,...,0,6,No,0,Unknown,No,3-Good,Town,Crafts,Yes
3,3000022,No,82.28,1312.0,75.0,1.24,0.0,0.0,157.0,8.1,...,0,6,No,0,10,No,4-Medium,Other,Other,No
4,3000026,Yes,17.14,0.0,17.0,0.00,0.0,0.0,0.0,-0.2,...,0,9,No,1,10,No,1-Highest,Other,Professional,Yes


In [4]:
df.columns

Index(['CustomerID', 'Churn', 'MonthlyRevenue', 'MonthlyMinutes',
       'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes',
       'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues',
       'DroppedCalls', 'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls',
       'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls',
       'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls',
       'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService',
       'UniqueSubs', 'ActiveSubs', 'ServiceArea', 'Handsets', 'HandsetModels',
       'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'ChildrenInHH',
       'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner',
       'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers',
       'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard',
       'RetentionCalls', 'RetentionOffersAccepted', 'NewCellphoneUser',
       'NotNewCellphoneUser', 'ReferralsMadeBySubscriber', 'IncomeGro

In [5]:
df.shape

(51047, 58)

# EDA

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51047 entries, 0 to 51046
Data columns (total 58 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CustomerID                 51047 non-null  int64  
 1   Churn                      51047 non-null  object 
 2   MonthlyRevenue             50891 non-null  float64
 3   MonthlyMinutes             50891 non-null  float64
 4   TotalRecurringCharge       50891 non-null  float64
 5   DirectorAssistedCalls      50891 non-null  float64
 6   OverageMinutes             50891 non-null  float64
 7   RoamingCalls               50891 non-null  float64
 8   PercChangeMinutes          50680 non-null  float64
 9   PercChangeRevenues         50680 non-null  float64
 10  DroppedCalls               51047 non-null  float64
 11  BlockedCalls               51047 non-null  float64
 12  UnansweredCalls            51047 non-null  float64
 13  CustomerCareCalls          51047 non-null  flo

 #   Column                     Non-Null Count  Dtype     Description 
---  ------                     --------------  -----     --------
 0   CustomerID                 51047 non-null  int64     Unique ID of customer
 1   Churn                      51047 non-null  object    Target column which tells whether customer churned or not
 2   MonthlyRevenue             50891 non-null  float64   Monthly revenue of customer
 3   MonthlyMinutes             50891 non-null  float64   Monthly minutes used by customer
 4   TotalRecurringCharge       50891 non-null  float64   Total recurring charges on bill
 5   DirectorAssistedCalls      50891 non-null  float64   Number of operator-assisted long-distance telephone call
 6   OverageMinutes             50891 non-null  float64   Extra minutes used beyond the limit included in their calling plan.
 7   RoamingCalls               50891 non-null  float64   Duration of roaming calls
 8   PercChangeMinutes          50680 non-null  float64   Call usage (in min) changes wrt to previous period.((C-P)/P)*100. Indicate changes in usage behavior
 9   PercChangeRevenues         50680 non-null  float64   Customer’s billed revenue has changed compared to a previous period.((C-P)/P) * 100. Revenue trend
 10  DroppedCalls               51047 non-null  float64   Duration of calls dropped. High values - network or service issues; affects customer satisfaction
 11  BlockedCalls               51047 non-null  float64   Duration of calls that were blocked
 12  UnansweredCalls            51047 non-null  float64   Duration of calls attempted but not answered
 13  CustomerCareCalls          51047 non-null  float64   Duration of Customer Care calls
 14  ThreewayCalls              51047 non-null  float64   Duration of three way conference calls
 15  ReceivedCalls              51047 non-null  float64   Duration of received calls
 16  OutboundCalls              51047 non-null  float64   Duration of outbound calls. Measures customer-initiated activity
 17  InboundCalls               51047 non-null  float64   Duration of inbound calls
 18  PeakCallsInOut             51047 non-null  float64   Total number of calls (incoming + outgoing) during peak hours
 19  OffPeakCallsInOut          51047 non-null  float64   Total number of calls (incoming + outgoing) during off peak hours
 20  DroppedBlockedCalls        51047 non-null  float64   Total duration of calls that were either dropped or blocked.
 21  CallForwardingCalls        51047 non-null  float64   Number of calls forwarded to another number
 22  CallWaitingCalls           51047 non-null  float64   Number of calls using call waiting, Indicates use of advanced calling features
 23  MonthsInService            51047 non-null  int64     Number of months the customer has been with the service, longer tenure often reduces churn risk
 24  UniqueSubs                 51047 non-null  int64     Number of unique subscriptions held by customer
 25  ActiveSubs                 51047 non-null  int64     Number of active subscriptions, shows current service usage
 26  ServiceArea                51023 non-null  object    Geographical area of customer’s service. Could relate to network quality or coverage differences
 27  Handsets                   51046 non-null  float64   Number of handsets owned. Customer equipment usage; may relate to upgrades or churn risk
 28  HandsetModels              51046 non-null  float64   Number of different handset models customer owns
 29  CurrentEquipmentDays       51046 non-null  float64   Days since the current handset was activated. Age of handset; older devices may correlate with churn
 30  AgeHH1                     50138 non-null  float64   Age of primary household member. Demographic information
 31  AgeHH2                     50138 non-null  float64   Age of secondary household member. Demographic information
 32  ChildrenInHH               51047 non-null  object    Indicator of children in household
 33  HandsetRefurbished         51047 non-null  object    Whether the handset is refurbished
 34  HandsetWebCapable          51047 non-null  object    Whether the handset can access the web
 35  TruckOwner                 51047 non-null  object    Whether the customer owns a truck. Lifestyle indicator
 36  RVOwner                    51047 non-null  object    Whether the customer owns a recreational vehicle. Lifestyle indicator
 37  Homeownership              51047 non-null  object    Own vs. rent
 38  BuysViaMailOrder           51047 non-null  object    Whether the customer buys via mail
 39  RespondsToMailOffers       51047 non-null  object    Whether customer responds to mail offers
 40  OptOutMailings             51047 non-null  object    Whether customer opted out of mailings
 41  NonUSTravel                51047 non-null  object    Whether customer travels outside US
 42  OwnsComputer               51047 non-null  object    Whether the customer owns a computer
 43  HasCreditCard              51047 non-null  object    Whether the customer has a credit card
 44  RetentionCalls             51047 non-null  int64     Number of calls from retention team
 45  RetentionOffersAccepted    51047 non-null  int64     Number of retention offers accepted
 46  NewCellphoneUser           51047 non-null  object    Whether the customer is a new cellphone user
 47  NotNewCellphoneUser        51047 non-null  object    Complement of above
 48  ReferralsMadeBySubscriber  51047 non-null  int64     Number of referrals made, Loyalty indicator; customer advocacy
 49  IncomeGroup                51047 non-null  int64     Income bracket / group of customer, socio economic indicator
 50  OwnsMotorcycle             51047 non-null  object    Whether customer owns a motorcycle,Lifestyle / demographic indicator
 51  AdjustmentsToCreditRating  51047 non-null  int64     Number of credit rating adjustments, Financial history; risk indicator
 52  HandsetPrice               51047 non-null  object    Price category of handset, socio economic indicator
 53  MadeCallToRetentionTeam    51047 non-null  object    Whether customer called retention team, Engagement / churn indicator
 54  CreditRating               51047 non-null  object    Credit rating category, Financial profile of customer
 55  PrizmCode                  51047 non-null  object    PRIZM segmentation code, marketing or lifestyle segmentation
 56  Occupation                 51047 non-null  object    Occupation category, demographic segmentation
 57  MaritalStatus              51047 non-null  object    Marital status, demgraphic segmentation

In [7]:
# Remove Customer ID
df.drop(columns='CustomerID', axis = 1, inplace=True)

In [8]:
df.columns

Index(['Churn', 'MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge',
       'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls',
       'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls',
       'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls',
       'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut',
       'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls',
       'CallWaitingCalls', 'MonthsInService', 'UniqueSubs', 'ActiveSubs',
       'ServiceArea', 'Handsets', 'HandsetModels', 'CurrentEquipmentDays',
       'AgeHH1', 'AgeHH2', 'ChildrenInHH', 'HandsetRefurbished',
       'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership',
       'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings',
       'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'RetentionCalls',
       'RetentionOffersAccepted', 'NewCellphoneUser', 'NotNewCellphoneUser',
       'ReferralsMadeBySubscriber', 'IncomeGroup', 'OwnsMoto

In [28]:
df.describe()

,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,...,Handsets,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,RetentionCalls,RetentionOffersAccepted,ReferralsMadeBySubscriber,IncomeGroup,AdjustmentsToCreditRating
count,50891.000000,50891.000000,50891.000000,50891.000000,50891.000000,50891.000000,50680.000000,50680.000000,51047.000000,51047.000000,...,51046.000000,51046.000000,51046.000000,50138.000000,50138.000000,51047.000000,51047.000000,51047.000000,51047.000000,51047.000000
mean,58.834492,525.653416,46.830088,0.895229,40.027785,1.236244,-11.547908,-1.191985,6.011489,4.085672,...,1.805646,1.558751,380.545841,31.338127,21.144142,0.037201,0.018277,0.052070,4.324524,0.053911
std,44.507336,529.871063,23.848871,2.228546,96.588076,9.818294,257.514772,39.574915,9.043955,10.946905,...,1.331173,0.905932,253.801982,22.094635,23.931368,0.206483,0.142458,0.307592,3.138236,0.383147
min,-6.170000,0.000000,-11.000000,0.000000,0.000000,0.000000,-3875.000000,-1107.700000,0.000000,0.000000,...,1.000000,1.000000,-5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,33.610000,158.000000,30.000000,0.000000,0.000000,0.000000,-83.000000,-7.100000,0.700000,0.000000,...,1.000000,1.000000,205.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,48.460000,366.000000,45.000000,0.250000,3.000000,0.000000,-5.000000,-0.300000,3.000000,1.000000,...,1.000000,1.000000,329.000000,36.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000
75%,71.065000,723.000000,60.000000,0.990000,41.000000,0.300000,66.000000,1.600000,7.700000,3.700000,...,2.000000,2.000000,515.000000,48.000000,42.000000,0.000000,0.000000,0.000000,7.000000,0.000000
max,1223.380000,7359.000000,400.000000,159.390000,4321.000000,1112.400000,5192.000000,2483.500000,221.700000,384.300000,...,24.000000,15.000000,1812.000000,99.000000,99.000000,4.000000,3.000000,35.000000,9.000000,25.000000


In [23]:
a = df.describe()

In [25]:
a = dict(a.iloc[1, :])

In [29]:
# Get the average values of all the columns to understand them better
a

{'MonthlyRevenue': np.float64(58.834492346387385),
 'MonthlyMinutes': np.float64(525.6534161246586),
 'TotalRecurringCharge': np.float64(46.830087834784145),
 'DirectorAssistedCalls': np.float64(0.895229018883496),
 'OverageMinutes': np.float64(40.02778487355328),
 'RoamingCalls': np.float64(1.2362441296103437),
 'PercChangeMinutes': np.float64(-11.547908445146014),
 'PercChangeRevenues': np.float64(-1.19198500394633),
 'DroppedCalls': np.float64(6.011489411718612),
 'BlockedCalls': np.float64(4.085672027739142),
 'UnansweredCalls': np.float64(28.28898074323662),
 'CustomerCareCalls': np.float64(1.8689991576390386),
 'ThreewayCalls': np.float64(0.29883832546476774),
 'ReceivedCalls': np.float64(114.80012145669679),
 'OutboundCalls': np.float64(25.37771465512175),
 'InboundCalls': np.float64(8.1781044919388),
 'PeakCallsInOut': np.float64(90.54951515270243),
 'OffPeakCallsInOut': np.float64(67.6507904480185),
 'DroppedBlockedCalls': np.float64(10.158003408623426),
 'CallForwardingCalls'

# Missing Data Analysis

In [30]:
# Check for missing values
df.isnull().sum()

Churn                          0
MonthlyRevenue               156
MonthlyMinutes               156
TotalRecurringCharge         156
DirectorAssistedCalls        156
OverageMinutes               156
RoamingCalls                 156
PercChangeMinutes            367
PercChangeRevenues           367
DroppedCalls                   0
BlockedCalls                   0
UnansweredCalls                0
CustomerCareCalls              0
ThreewayCalls                  0
ReceivedCalls                  0
OutboundCalls                  0
InboundCalls                   0
PeakCallsInOut                 0
OffPeakCallsInOut              0
DroppedBlockedCalls            0
CallForwardingCalls            0
CallWaitingCalls               0
MonthsInService                0
UniqueSubs                     0
ActiveSubs                     0
ServiceArea                   24
Handsets                       1
HandsetModels                  1
CurrentEquipmentDays           1
AgeHH1                       909
AgeHH2    

In [40]:
for col in df.columns:
    if df[col].isnull().sum() >0:
        print(f'{col:<25} contains missing values - {df[col].isnull().sum():5}, type - {df[col].dtype}')

MonthlyRevenue            contains missing values -   156, type - float64
MonthlyMinutes            contains missing values -   156, type - float64
TotalRecurringCharge      contains missing values -   156, type - float64
DirectorAssistedCalls     contains missing values -   156, type - float64
OverageMinutes            contains missing values -   156, type - float64
RoamingCalls              contains missing values -   156, type - float64
PercChangeMinutes         contains missing values -   367, type - float64
PercChangeRevenues        contains missing values -   367, type - float64
ServiceArea               contains missing values -    24, type - object
Handsets                  contains missing values -     1, type - float64
HandsetModels             contains missing values -     1, type - float64
CurrentEquipmentDays      contains missing values -     1, type - float64
AgeHH1                    contains missing values -   909, type - float64
AgeHH2                    contains miss

# Unique Count

In [38]:
for col in df.columns:
    print(f'{col:<25} has {df[col].nunique():>10} values ')

Churn                     has          2 values 
MonthlyRevenue            has      12665 values 
MonthlyMinutes            has       2719 values 
TotalRecurringCharge      has        214 values 
DirectorAssistedCalls     has        146 values 
OverageMinutes            has        746 values 
RoamingCalls              has        530 values 
PercChangeMinutes         has       2262 values 
PercChangeRevenues        has       2899 values 
DroppedCalls              has        285 values 
BlockedCalls              has        373 values 
UnansweredCalls           has        840 values 
CustomerCareCalls         has        185 values 
ThreewayCalls             has         78 values 
ReceivedCalls             has       6121 values 
OutboundCalls             has        787 values 
InboundCalls              has        472 values 
PeakCallsInOut            has       1839 values 
OffPeakCallsInOut         has       1669 values 
DroppedBlockedCalls       has        447 values 
CallForwardingCalls 

In [45]:
print('Columns with less than or equal to 10 unique values')
for col in df.columns:
    if df[col].nunique()<=10:
        print(f'{col:<25} has {df[col].nunique():>10} values ')
print('***************************************')
print('***************************************')
print('Columns with more than 10 unique values')
for col in df.columns:
    if df[col].nunique()>10:
        print(f'{col:<25} has {df[col].nunique():>10} values')

Columns with less than or equal to 10 unique values
Churn                     has          2 values 
ChildrenInHH              has          2 values 
HandsetRefurbished        has          2 values 
HandsetWebCapable         has          2 values 
TruckOwner                has          2 values 
RVOwner                   has          2 values 
Homeownership             has          2 values 
BuysViaMailOrder          has          2 values 
RespondsToMailOffers      has          2 values 
OptOutMailings            has          2 values 
NonUSTravel               has          2 values 
OwnsComputer              has          2 values 
HasCreditCard             has          2 values 
RetentionCalls            has          5 values 
RetentionOffersAccepted   has          4 values 
NewCellphoneUser          has          2 values 
NotNewCellphoneUser       has          2 values 
IncomeGroup               has         10 values 
OwnsMotorcycle            has          2 values 
MadeCallToRetenti

In [47]:
print('Columns with less than or equal to 10 unique values')
cat = []
num = []
for col in df.columns:
    if df[col].nunique()<=10:
        cat.append(col)
print('***************************************')
print('***************************************')
print('Columns with more than 10 unique values')
for col in df.columns:
    if df[col].nunique()>10:
        num.append(col)

print(cat)
print('-----')
print(num)

Columns with less than or equal to 10 unique values
***************************************
***************************************
Columns with more than 10 unique values
['Churn', 'ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'RetentionCalls', 'RetentionOffersAccepted', 'NewCellphoneUser', 'NotNewCellphoneUser', 'IncomeGroup', 'OwnsMotorcycle', 'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode', 'Occupation', 'MaritalStatus']
-----
['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls', 'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls', 'CallW

# Duplicate values check

In [46]:
df.duplicated().sum()

np.int64(0)

No duplicates are present

# Categorise data into numerical and categorical type for analysis
- If a columns has 10 or less unique values then it is categorical.
- If a column has more than 10 unique values then it is numerical however some columns like ServiceArea although have more than 10 unique values, will be considered as categorical. 


In [ ]:
categorical_features = ['ChildrenInHH', 'HandsetRefurbished', 'HandsetWebCapable', 'TruckOwner', 'RVOwner', 'Homeownership', 'BuysViaMailOrder', 'RespondsToMailOffers', 'OptOutMailings', 'NonUSTravel', 'OwnsComputer', 'HasCreditCard', 'RetentionCalls', 'RetentionOffersAccepted', 'NewCellphoneUser', 'NotNewCellphoneUser', 'IncomeGroup', 'OwnsMotorcycle', 'MadeCallToRetentionTeam', 'CreditRating', 'PrizmCode', 'Occupation', 'MaritalStatus', 'ServiceArea', ]

numerical_features = ['MonthlyRevenue', 'MonthlyMinutes', 'TotalRecurringCharge', 'DirectorAssistedCalls', 'OverageMinutes', 'RoamingCalls', 'PercChangeMinutes', 'PercChangeRevenues', 'DroppedCalls', 'BlockedCalls', 'UnansweredCalls', 'CustomerCareCalls', 'ThreewayCalls', 'ReceivedCalls', 'OutboundCalls', 'InboundCalls', 'PeakCallsInOut', 'OffPeakCallsInOut', 'DroppedBlockedCalls', 'CallForwardingCalls', 'CallWaitingCalls', 'MonthsInService', 'UniqueSubs', 'ActiveSubs', 'Handsets', 'CurrentEquipmentDays', 'AgeHH1', 'AgeHH2', 'ReferralsMadeBySubscriber', 'AdjustmentsToCreditRating', 'HandsetPrice', 'HandsetModels']

target = ['Churn']


# Convert datatypes from object to int/float to feed data into model

In [ ]:
# List of categorical features
object_features = [col for col in df.columns if df[col].dtype == 'object']
print(object_features)

In [ ]:
def convert_object_data_into_int_float(df, smoothing_factor=10, handle_na = True):
    # handle_na is used for treating missing values as separate category in ServiceArea. 
    # If handle_na is True then the NAN values in ServiceArea will be treated as a separate category otherwise these values will be replaced by global mean. 

    binary_mappings = {
        'Churn': {'No':0, 'Yes': 1},
        'ChildrenInHH': {'No':0, 'Yes':1},
        'HandsetRefurbished': {'No':0, 'Yes':1},
        'HandsetWebCapable': {'No':0, 'Yes': 1},
        'TruckOwner': {'No': 0, 'Yes':1},
        'RVOwner': {'No':0, 'Yes': 1},
        'Homeownership': {'Unknown': 0, 'Known':1},
        'BuysViaMailOrder': {'No':0, 'Yes':1},
        'RespondsToMailOffers': {'No':0, 'Yes':1},
        'OptOutMailings': {'No':0, 'Yes':1},
        'NonUSTravel': {'No':0, 'Yes':1},
        'OwnsComputer': {'No':0, 'Yes': 1},
        'HasCreditCard': {'No': 0, 'Yes': 1},
        'NewCellphoneUser': {'No':0, 'Yes':1},
        'NotNewCellphoneUser': {'No':0, 'Yes':1},
        'OwnsMotorcycle': {'No':0, 'Yes':1},
        'MadeCallToRetentionTeam': {'No': 0, 'Yes':1}
    }
    # Apply binary mappings
    for col, mapping in binary_mappings.items():
        # col = col + '_num'
        df[col] = df[col].map(mapping)
    
    # for multi category columns, we are going to use pandas get_dummies. 
    multi_category_columns = ['CreditRating', 'PrizmCode', 'Occupation', 'MaritalStatus']
    # Creating a separate dataframe where I am storing all these dummies that are created. Then I am concatenating this with original df to keep the original column too.
    dummies = pd.get_dummies(df[multi_category_columns], drop_first=True)
    df_with_dummies = pd.concat([df, dummies], axis = 1)
    df_with_dummies.drop(multi_category_columns, axis=1, inplace=True)
    # Now ServiceArea has more than 20 categories, for this I am using target encoding with cross validation
    # Create a separate column where we will store the target encoding for Service Area column
    df_with_dummies['ServiceArea_te'] = np.nan

    if handle_na == True:
        df_with_dummies['ServiceArea'] = df_with_dummies['ServiceArea'].fillna('Missing')

    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    m = smoothing_factor # Smoothing factor
    global_mean = df_with_dummies['Churn'].mean()

    for train_idx, val_idx in kf.split(df_with_dummies):
        train_fold = df_with_dummies.iloc[train_idx]
        ## val_fold = df_with_dummies.iloc[val_idx]

        # Compute mean and count for target for each category in training fold
        stats = train_fold.groupby('ServiceArea')['Churn'].agg(['mean', 'count'])
        
        # apply smoothing
        stats['smoothed'] = ((stats['count'] * stats['mean'] + m * global_mean))/ (stats['count'] + m)

        # Map smoothed values to validation set
        df_with_dummies.loc[val_idx, 'ServiceArea_te'] = df_with_dummies.loc[val_idx, 'ServiceArea'].map(stats['smoothed'])
    
    # Fill unseen categories with global mean
    df_with_dummies['ServiceArea_te'] = df_with_dummies['ServiceArea_te'].fillna(global_mean)
    # Store the final results in json file for use in inference

    stats = df_with_dummies.groupby('ServiceArea')['Churn'].agg(['mean', 'count'])
    stats['TE'] = ((stats['mean'] * stats['count']) + (global_mean * smoothing_factor)) / (stats['count'] + smoothing_factor)
    final_encoding = stats['TE'].to_dict()
    with open('service_area_encoding.json', 'w') as f:
        json.dump(final_encoding, f)
    df_with_dummies.drop(['ServiceArea'], axis = 1, inplace=True)
    # HandsetPrice is also a column with object datatype but that is because of the presence of missing values. When we handle these missing values then the column would automatically become numerical type.
    return df_with_dummies

    



In [ ]:
# Now we just call the above function to convert object datatype columns into numerical datatype columns
df_numerical= convert_object_data_into_int_float(df)

In [ ]:
df_numerical.shape

In [ ]:
df_numerical.info()

# Handle missing values

In [ ]:
# Hanlding missing value in MonthlyRevenue
# Using KNN imputation. 
# Reason to use KNN imputation is because the missingness of data can be explained by other features in the dataset. 
# Target - MonthlyRevenue
# Independent features - Monthlyminutes, TotalRecurringCharge, DirectorAssistedCalls, OverageMinutes, RoamingCalls, PercChangeMinutes, PeakCallsInOut, OffPeakCallsInOut, ServiceArea, Homeownership, RetentionCalls, RetentionOffersAccepted, IncomeGroup

# Notes

1. CustomerID does not add any value in prediction. Removing it
2. Converting object data type into numerical data type. This is needed as we cannot give string data to model. For columns with only two categories, I have simply mapped the categories to 0 and 1. for columns with more than 5 but less than 10 categories, I have used one hot encoding using pandas get dummies. We cannot just assign values like 1, 2, 3 , 4 etc to these as then the model will treat this like a ordinal data when in fact it is not. Model might try to get some relation between these values when in fact it does not exist. Therefore we need to use one hot encoding. 
For column like service area with 100s of unique values there is a choice either we can use Frequency encoding or target encoding. Each has their own advantage and disadvantage. 
Frequency encoding - Replaces each category with its relative frequency in the dataset.
Pros:
 - Simple and safe (no risk of leakage).
- Works with any model (tree-based, linear, deep learning).
- Keeps dataset compact.
Cons:
- Doesn’t capture the relationship between category and churn.
- Two service areas with same frequency but very different churn rates will look identical.

Target encoding - Replaces each category with the average target value (churn probability) for that category.
Pros:
- Very powerful — directly encodes predictive information.
- Often improves performance, especially with high-cardinality features.
Cons:
- Data leakage risk if not done carefully → must apply encoding inside cross-validation folds.
- Can overfit if a category has very few samples.

Smoothed target encoding formula
For each category c,
Target encoding (C) = ((nc * mean(c)) + (m * global_mean))/(nc + m)
mean(c) = average churn in category c
nc = number of samples in category c
global_mean = overall churn rate
m  = smoothing factor (higher = more weight on global mean)

I am going to be using target encoding and in that I am also using cross validation to avoid data leakage. 

Readme
Target columns is Churn
Perform EDA - Understand all the columns and meaning of data represented by each column
Convert all object data type columns to numerical form

In [ ]:
df_test = df.copy()

In [ ]:
df_test = pd.get_dummies(df_test, columns = ['CreditRating'], drop_first=True)

In [ ]:
df_test.columns

In [ ]:
df_test.head()